In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("../google.csv")
df

FileNotFoundError: [Errno 2] File ../google.csv does not exist: '../google.csv'

In [4]:
df.nunique()

id             1561206
title          1520344
description    1228733
category          5492
images         1510682
seller           41507
url            1561206
dtype: int64

In [5]:
id_sorted = df.sort_values(["id"])[:100]
print(id_sorted[["id", "title"]].to_string())

                           id                                                                                                                                       title
1220422  10000000167364349403                                                                                       Summit Buddha Amitabha Collectible Sculpture EZ268303
304792   10000000167364349403                                                                                       Summit Buddha Amitabha Collectible Sculpture EZ268303
1025288  10000007198605757577  Ekena Millwork 14 1/2 inchw x 53 inchh Lifetime Vinyl, Custom Offset Panel, Raised Panel Shutters, w/Shutter-Loks (Per Pair), Classic Blue
1517063   1000001423483945629                                                                DEYIOU 48 Color Pencil Premium Pre-Sharpened Water Soluble Water Crayons Set
990665   10000021515066185429                                                                 Royal Langnickel Majestic R4100T-16 Taklon Acrylic and O

In [6]:
df = df[~df.duplicated(subset=["id"])]
df = df[~df.duplicated(subset=["title"])]

In [7]:
len(df)

1518054

In [8]:
df.isna().sum()

id                 0
title              0
description    34468
category           0
images             0
seller             0
url                0
dtype: int64

## Remapping

In [20]:
remapping = pd.read_csv("../google_category_remapping.csv")
del remapping['counts']  # incorrect counts
remapping

,index,category,L1,L2
0,11,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
1,227,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
2,13,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
3,416,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
4,236,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
...,...,...,...,...
1333,431,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1334,839,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1335,578,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1336,424,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories


In [21]:
low_confidence_index = [915, 325, 652, 684, 514, 749, 355, 314, 992, 941, 517, 1240, 552, 604, 405, 1271, 877, 643, 970, 1030, 1252, 1191, 688, 700, 348, 278]

remapping = remapping.loc[~remapping["index"].isin(low_confidence_index), :]
remapping

,index,category,L1,L2
0,11,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
1,227,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
2,13,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
3,416,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
4,236,Home > Baby & Kids > Baby > Baby & Toddler Toy...,Baby & Child,Toys
...,...,...,...,...
1333,431,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1334,839,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1335,578,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories
1336,424,Home > Pet Supplies > Small Animal Supplies > ...,Pet Care,Pet Accessories


In [22]:
remapping.isna().sum()

index       0
category    0
L1          0
L2          0
dtype: int64

In [23]:
def clean_string(string):
    string = " ".join(string.split(" "))
    string = string.rstrip().lstrip()
    return string

In [24]:
remapping.columns

Index(['index', 'category', 'L1', 'L2'], dtype='object')

In [25]:
remapping.loc[:, "L1"] = remapping["L1"].apply(clean_string)
remapping.loc[:, "L2"] = remapping["L2"].apply(clean_string)
remapping.loc[:, "category"] = remapping["category"].apply(clean_string)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [26]:
print(sum(remapping["L1"] == "MULTIMATCH"))
print(sum(remapping["L2"] == "MULTIMATCH"))
print(sum((remapping[["L1", "L2"]] == "MULTIMATCH").any(1)))

print(sum(remapping["L1"] == "OTHER"))
print(sum(remapping["L2"] == "OTHER"))
print(sum((remapping[["L1", "L2"]] == "OTHER").any(1)))

0
3
3
2
171
171


In [27]:
remapping = remapping[~(remapping[["L1", "L2"]].isin(["MULTIMATCH", "OTHER"])).any(1)]
len(remapping)

1138

In [54]:
category_mapping = {}
for i, row in remapping.iterrows():
    category_mapping[row["category"]] = (row["L1"], row["L2"])

In [55]:
def get_section_categories(section):
    # last_parts = section.split(">")[-2:]
    # key = ">".join(section).lstrip().rstrip()
    result = category_mapping.get(section.lstrip().rstrip(), (np.nan, np.nan))
    return result

In [56]:
"Home > Apparel > Clothing > Shirts & Tops > Polo Shirts " in category_mapping

False

In [57]:
df["L1"] = df["category"].apply(lambda x: get_section_categories(x)[0])
df["L2"] = df["category"].apply(lambda x: get_section_categories(x)[1])
df.head()

,id,title,description,category,images,seller,url,L1,L2
0,600461809799217547,Safavieh Courtyard Area Rug CY2720 Natural / O...,"Safavieh Courtyard 2' 3"" X 10' Indoor/outdoor ...",Home > Home & Garden > Home Decor > Rugs & Doo...,['https://encrypted-tbn3.gstatic.com/shopping?...,Home Depot,https://www.google.com/shopping/product/600461...,NaN,NaN
1,5577337335524048579,Innerspace RV & Truck Luxury Deluxe 8-inch Mem...,The MotorHome InnerSpace Luxury Deluxe 8 in. M...,Home > Home & Garden > Bedding & Bath > Beddin...,['https://encrypted-tbn3.gstatic.com/shopping?...,Overstock.com,https://www.google.com/shopping/product/557733...,NaN,NaN
2,14258831033128011323,willstar10Pcs/set Natural Wooden Exercise Bell...,Specification: Material: natural wood Size: as...,Home > Pet Supplies > Small Animal Supplies > ...,['https://encrypted-tbn3.gstatic.com/shopping?...,Walmart - Joybuy,https://www.google.com/shopping/product/142588...,Pet Care,Pet Accessories
3,7485254319593032123,Superior 300 Thread Count Combed Cotton Sateen...,Woven from 100% Premium Long-Staple Combed Cot...,Home > Home & Garden > Bedding & Bath > Beddin...,['https://encrypted-tbn3.gstatic.com/shopping?...,Walmart - UnbeatableSale,https://www.google.com/shopping/product/748525...,NaN,NaN
4,11558129222548876322,Satco S5822 HID Metal Halide ED28,Get the perfect light with the ED28 EX39 Mogul...,Home > Home & Garden > Lighting > Lighting Par...,['https://encrypted-tbn3.gstatic.com/shopping?...,Topbulb,https://www.google.com/shopping/product/115581...,NaN,NaN


In [58]:
sum(pd.isnull(df["L1"]))

1321336

In [63]:
print(df[:200][["title", "category"]].to_string())

                                                                                                                                                                             title                                                                                                                                                                                         category
0                                                                                                                  Safavieh Courtyard Area Rug CY2720 Natural / Olive, 2' 3" x 10'                                                                                                                        Home > Home & Garden > Home Decor > Rugs & Doormats > Rugs > Outdoor Rugs
1                                                                                                           Innerspace RV & Truck Luxury Deluxe 8-inch Memory Foam Mattress, White                                                                              

Compare to doordash

In [4]:
doordash = pd.read_csv("doordash.csv")
doordash

FileNotFoundError: [Errno 2] File doordash.csv does not exist: 'doordash.csv'

In [ ]:
doordash.groupby(["L1", "L2"]).size()